The notebook aims to highlight **`the ease of using autoML on containers to solve a common classification problem`**
- Loads data to CAS
- Runs AutoML (datasciencepilot) on CAS
- Generates the champ model
- Scores data with champ model
- Saves scoring results & model to the filesystem
- Spawns h2o session
- Runs h2o automl against the data
- Generates the champ model
- Optionally,saves the champ model 

In [163]:
import swat,pygal
from _config import container_cas35_login
from swat.render import render_html
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)
from matplotlib import pyplot as plt
from IPython.display import HTML
%matplotlib inline
# from plotnine import *
import os, time
#os.environ['CAS_CLIENT_SSL_CA_LIST']=r'/ext_str/beast_trustfiles/nBeast_trustfiles/trustedcerts.pem'
from IPython.display import IFrame
%reload_ext autoreload
%autoreload 2

In [164]:
host = container_cas35_login()[2]
user = container_cas35_login()[0]
pswd = container_cas35_login()[1]

In [165]:
sess = swat.CAS(host,5571,user,pswd)
sess.setsessopt(caslib="casuser")
sess.loadactionset(actionset="dataSciencePilot")

NOTE: 'CASUSER(sasdemo)' is now the active caslib.
NOTE: Added action set 'dataSciencePilot'.


[actionset]

 'dataSciencePilot'

+ Elapsed: 0.00586s, user: 0.00284s, sys: 0.00294s, mem: 0.22mb

In [167]:
swat.options.tkpath

'/home/centos/.conda/envs/dl4viya/lib/python3.7/site-packages/swat/lib/linux'

In [126]:
sess.help(actionset="dataSciencePilot")["dataSciencePilot"]

NOTE: Information for action set 'dataSciencePilot':
NOTE:    dataSciencePilot
NOTE:       exploreData - Exploration, automatic variable analysis and grouping using comprehensive statistical profiling of the variables.
NOTE:       screenVariables - Screens noise variables and variables that need special transformations to be useful in the downstream analytics.
NOTE:       analyzeMissingPatterns - Missing pattern analysis
NOTE:       exploreCorrelation - Explore linear and nonlinear correlation among the variables.
NOTE:       detectInteractions - Variable interaction detection and ranking
NOTE:       generateShadowFeatures - Generate shadow features.
NOTE:       featureMachine - Automated feature transformation and generation engine
NOTE:       selectFeatures - Feature selection
NOTE:       dsAutoMl - Automated machine learning pipeline exploration, execution and ranking.


,name,description
0,exploreData,"Exploration, automatic variable analysis and g..."
1,screenVariables,Screens noise variables and variables that nee...
2,analyzeMissingPatterns,Missing pattern analysis
3,exploreCorrelation,Explore linear and nonlinear correlation among...
4,detectInteractions,Variable interaction detection and ranking
5,generateShadowFeatures,Generate shadow features.
6,featureMachine,Automated feature transformation and generatio...
7,selectFeatures,Feature selection
8,dsAutoMl,Automated machine learning pipeline exploratio...


In [127]:
churn_df=pd.read_csv('churn.csv')
churn_df.columns = [i.replace(' ','_').replace("'",'').lower() for i in churn_df.columns]
#Check out how the resultant dataset look like
churn_df.head()

,account_length,vmail_message,day_mins,eve_mins,night_mins,intl_mins,custserv_calls,churn,intl_plan,vmail_plan,...,day_charge,eve_calls,eve_charge,night_calls,night_charge,intl_calls,intl_charge,state,area_code,phone
0,128,25,265.10,197.40,244.70,10.00,1,0,0,1,...,45.07,99,16.78,91,11.01,3,2.70,KS,415,382-4657
1,107,26,161.60,195.50,254.40,13.70,1,0,0,1,...,27.47,103,16.62,103,11.45,3,3.70,OH,415,371-7191
2,137,0,243.40,121.20,162.60,12.20,0,0,0,0,...,41.38,110,10.30,104,7.32,5,3.29,NJ,415,358-1921
3,84,0,299.40,61.90,196.90,6.60,2,0,1,0,...,50.90,88,5.26,89,8.86,7,1.78,OH,408,375-9999
4,75,0,166.70,148.30,186.90,10.10,3,0,1,0,...,28.34,122,12.61,121,8.41,3,2.73,OK,415,330-6626


In [128]:
# Load Table to CAS
out=sess.upload(churn_df,casout=dict(name='churn',caslib='casuser'))
sess.loadactionset('fedSQL') #Enable SQL actions - for distributed SQL         
out=out.casTable #get CASTAble
#programmatically build query
col_list= [i for i in out.columns if i not in ('area_code','churn','intl_plan','vmail_plan')]
cas_lib='casuser'
option_params='{options replace=true}'
query = """create table {}.churn {} as select {}, 
cast(intl_plan as char) as intl_plan,
cast(vmail_plan as char) as vmail_plan,
cast(area_code as varchar) as area_code,
cast(churn as varchar) as churn
from casuser.churn """.format(cas_lib,
                              option_params,
                              col_list).replace('[','').replace(']','').replace("'",'')

#execute query and check the results
sess.fedsql.execdirect(query) # run the query
out = sess.CASTable('CHURN', caslib ='casuser') #get the results
render_html(out.fetch(to=5)) #view the results

NOTE: Cloud Analytic Services made the uploaded file available as table CHURN in caslib CASUSER(sasdemo).
NOTE: The table CHURN has been created in caslib CASUSER(sasdemo) from binary data uploaded to Cloud Analytic Services.
NOTE: Added action set 'fedSQL'.
NOTE: Table CHURN was created in caslib CASUSER(sasdemo) with 3333 rows returned.


Selected Rows from Table CHURN 
 account_length 
 vmail_message 
 day_mins 
 eve_mins 
 night_mins 
 intl_mins 
 custserv_calls 
 day_calls 
 day_charge 
 eve_calls 
 eve_charge 
 night_calls 
 night_charge 
 intl_calls 
 intl_charge 
 state 
 phone 
 INTL_PLAN 
 VMAIL_PLAN 
 AREA_CODE 
 CHURN 
 
 
 128 25 265.1 197.4 244.7 10 1 110 45.07 99 16.78 91 11.01 3 2.7 KS 382-4657 0 1 415 0 
 107 26 161.6 195.5 254.4 13.7 1 123 27.47 103 16.62 103 11.45 3 3.7 OH 371-7191 0 1 415 0 
 137 0 243.4 121.2 162.6 12.2 0 114 41.38 110 10.3 104 7.32 5 3.29 NJ 358-1921 0 0 415 0 
 84 0 299.4 61.9 196.9 6.6 2 71 50.9 88 5.26 89 8.86 7 1.78 OH 375-9999 1 0 408 0 
 75 0 166.7 148.3 186.9 10.1 3 113 28.34 122 12.61 121 8.41 3 2.73 OK 330-6626 1 0 415 0

In [129]:
effect_vars={'area_code','account_length', 'custserv_calls', 'day_calls', 'day_charge', 
             'day_mins', 'eve_calls','eve_charge', 'eve_mins', 'intl_calls', 
             'intl_charge', 'intl_mins', 'intl_plan','night_calls','night_charge',
             'night_mins', 'vmail_message', 'vmail_plan'}

In [130]:
sess.dataSciencePilot.dsAutoMl(   
        table                   = out,
        target                  = "CHURN",
        inputs = effect_vars,
#         explorationPolicy       = {},
#         screenPolicy            = {},
#         selectionPolicy         = {},
        transformationPolicy    = {"missing":True, "cardinality":True,
                                   "entropy":True, "iqv":True,
                                   "skewness":True, "kurtosis":True, "Outlier":True},
        modelTypes              = ["decisionTree", "GRADBOOST"],
        objective               = "AUC",
        sampleSize              = 20,
        topKPipelines           = 10,
        kFolds                  = 2,
        transformationOut       = {"name" : "TRANSFORMATION_OUT", "replace" : True},
        featureOut              = {"name" : "FEATURE_OUT", "replace" : True},
        pipelineOut             = {"name" : "PIPELINE_OUT", "replace" : True},
        saveState               = dict(modelNamePrefix='churn_model', replace = True)      
    )

NOTE: Added action set 'autotune'.
NOTE: Added action set 'decisionTree'.
NOTE: Early stopping is activated; 'NTREE' will not be tuned.
NOTE: Added action set 'autotune'.
NOTE: Added action set 'decisionTree'.
NOTE: Early stopping is activated; 'NTREE' will not be tuned.
NOTE: Added action set 'autotune'.
NOTE: Added action set 'decisionTree'.
NOTE: Early stopping is activated; 'NTREE' will not be tuned.
NOTE: Added action set 'autotune'.
NOTE: Added action set 'decisionTree'.
NOTE: Early stopping is activated; 'NTREE' will not be tuned.
NOTE: Added action set 'autotune'.
NOTE: Added action set 'decisionTree'.
NOTE: Early stopping is activated; 'NTREE' will not be tuned.
NOTE: Added action set 'autotune'.
NOTE: Added action set 'decisionTree'.
NOTE: Early stopping is activated; 'NTREE' will not be tuned.
NOTE: Added action set 'autotune'.
NOTE: Added action set 'decisionTree'.
NOTE: Early stopping is activated; 'NTREE' will not be tuned.
NOTE: Added action set 'autotune'.
NOTE: Added a

,Descr,Value
0,Number of Tree Nodes,31.00
1,Max Number of Branches,2.00
2,Number of Levels,5.00
3,Number of Leaves,16.00
4,Number of Bins,100.00
5,Minimum Size of Leaves,5.00
6,Maximum Size of Leaves,2214.00
7,Number of Variables,12.00
8,Confidence Level for Pruning,0.25
9,Number of Observations Used,3333.00


In [131]:
sess.tableinfo()

,Name,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,CreateTime,...,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime
0,CHURN,3333,21,0,utf-8,2020-05-30T00:17:14+00:00,2020-05-30T00:17:14+00:00,2020-05-30T00:17:15+00:00,UTF8,1906417033.84,...,0,0,0,,,0,sasdemo,,,nan
1,CHURN_MODEL_FM_,1,2,0,utf-8,2020-05-30T00:17:16+00:00,2020-05-30T00:17:16+00:00,2020-05-30T00:23:17+00:00,UTF8,1906417035.60,...,0,0,0,,,0,sasdemo,,,nan
2,TRANSFORMATION_OUT,19,21,0,utf-8,2020-05-30T00:17:16+00:00,2020-05-30T00:17:16+00:00,2020-05-30T00:23:17+00:00,UTF8,1906417035.64,...,0,0,0,,,0,sasdemo,,,nan
3,FEATURE_OUT,106,15,0,utf-8,2020-05-30T00:17:16+00:00,2020-05-30T00:17:16+00:00,2020-05-30T00:23:17+00:00,UTF8,1906417035.64,...,0,0,0,,,0,sasdemo,,,nan
4,CHURN_MODEL_GRADBOOST_1,1,2,0,utf-8,2020-05-30T00:21:35+00:00,2020-05-30T00:21:35+00:00,2020-05-30T00:23:17+00:00,UTF8,1906417295.20,...,0,0,0,,,0,sasdemo,,,nan
5,CHURN_MODEL_GRADBOOST_2,1,2,0,utf-8,2020-05-30T00:22:08+00:00,2020-05-30T00:22:08+00:00,2020-05-30T00:23:17+00:00,UTF8,1906417327.62,...,0,0,0,,,0,sasdemo,,,nan
6,CHURN_MODEL_GRADBOOST_3,1,2,0,utf-8,2020-05-30T00:22:35+00:00,2020-05-30T00:22:35+00:00,2020-05-30T00:23:17+00:00,UTF8,1906417355.23,...,0,0,0,,,0,sasdemo,,,nan
7,CHURN_MODEL_GRADBOOST_4,1,2,0,utf-8,2020-05-30T00:22:55+00:00,2020-05-30T00:22:55+00:00,2020-05-30T00:23:17+00:00,UTF8,1906417374.70,...,0,0,0,,,0,sasdemo,,,nan
8,CHURN_MODEL_GRADBOOST_5,1,2,0,utf-8,2020-05-30T00:23:17+00:00,2020-05-30T00:23:17+00:00,2020-05-30T00:23:17+00:00,UTF8,1906417396.57,...,0,0,0,,,0,sasdemo,,,nan
9,PIPELINE_OUT,10,49,0,utf-8,2020-05-30T00:23:17+00:00,2020-05-30T00:23:17+00:00,2020-05-30T00:23:17+00:00,UTF8,1906417396.60,...,0,0,0,,,0,sasdemo,,,nan


In [132]:
churn_model_fm_ = sess.CASTable('CHURN_MODEL_FM_')

In [133]:
sess.loadactionset('astore')

NOTE: Added action set 'astore'.


[actionset]

 'astore'

+ Elapsed: 0.000865s, user: 0.000693s, mem: 0.207mb

In [134]:
sess.describe(churn_model_fm_)

[Key]

 Store Key
 
                                         Key
 0  49F889E861F08F261DE9EE6FC3EC92BAE97EDEC1

[Description]

 Basic Information
 
          Attribute               Value
 0  Analytic Engine                 fte
 1     Time Created  30May2020:00:17:15

[InputVariables]

 Input Variables
 
               Name  Length   Role      Type    RawType FormatName
 0       day_charge    8.00  Input  Interval        Num           
 1         day_mins    8.00  Input  Interval        Num           
 2       eve_charge    8.00  Input  Interval        Num           
 3         eve_mins    8.00  Input  Interval        Num           
 4      intl_charge    8.00  Input  Interval        Num           
 5        intl_mins    8.00  Input  Interval        Num           
 6     night_charge    8.00  Input  Interval        Num           
 7       night_mins    8.00  Input  Interval        Num           
 8   account_length    8.00  Input  Interval        Num           
 9   custserv_calls    8.00  Input  Interval        Num           
 10       day_calls    8.00  Input  Interval        Num           
 11       eve_calls    8.00  Input  Interval        Num           
 12      intl_calls    8.00  Input  Interval        Num           
 13     night_calls    8.00  Input  Interval        Num           
 14   vmail_message    8.00  Input  Interval        Num           
 15       INTL_PLAN    1.00  Input  Interval  Character           
 16      VMAIL_PLAN    1.00  Input  Interval  Character           

[OutputVariables]

 Output Variables
 
                                 Name  Length Type  \
 0    nhoks_nloks_dtree_10_day_charge    8.00  Num   
 1     nhoks_nloks_dtree_5_day_charge    8.00  Num   
 2         nhoks_nloks_log_day_charge    8.00  Num   
 3    nhoks_nloks_pow_n0_5_day_charge    8.00  Num   
 4      nhoks_nloks_dtree_10_day_mins    8.00  Num   
 5       nhoks_nloks_dtree_5_day_mins    8.00  Num   
 6           nhoks_nloks_log_day_mins    8.00  Num   
 7      nhoks_nloks_pow_p0_5_day_mins    8.00  Num   
 8    nhoks_nloks_dtree_10_eve_charge    8.00  Num   
 9     nhoks_nloks_dtree_5_eve_charge    8.00  Num   
 10   nhoks_nloks_pow_p0_5_eve_charge    8.00  Num   
 11     nhoks_nloks_pow_p2_eve_charge    8.00  Num   
 12          cpy_int_med_imp_eve_mins    8.00  Num   
 13     nhoks_nloks_dtree_10_eve_mins    8.00  Num   
 14      nhoks_nloks_dtree_5_eve_mins    8.00  Num   
 15       nhoks_nloks_pow_p1_eve_mins    8.00  Num   
 16       nhoks_nloks_pow_p2_eve_mins    8.00  Num   
 17  nhoks_nloks_dtree_10_intl_charge    8.00  Num   
 18   nhoks_nloks_dtree_5_intl_charge    8.00  Num   
 19       nhoks_nloks_log_intl_charge    8.00  Num   
 20  nhoks_nloks_pow_p0_5_intl_charge    8.00  Num   
 21    nhoks_nloks_dtree_10_intl_mins    8.00  Num   
 22     nhoks_nloks_dtree_5_intl_mins    8.00  Num   
 23    nhoks_nloks_pow_p0_5_intl_mins    8.00  Num   
 24      nhoks_nloks_pow_p2_intl_mins    8.00  Num   
 25       nhoks_nloks_dtree_10_var_2_    8.00  Num   
 26  nhoks_nloks_dtree_5_night_charge    8.00  Num   
 27       nhoks_nloks_pow_n0_5_var_2_    8.00  Num   
 28   nhoks_nloks_pow_n2_night_charge    8.00  Num   
 29        cpy_int_med_imp_night_mins    8.00  Num   
 30   nhoks_nloks_dtree_10_night_mins    8.00  Num   
 31    nhoks_nloks_dtree_5_night_mins    8.00  Num   
 32     nhoks_nloks_pow_p1_night_mins    8.00  Num   
 33     nhoks_nloks_pow_p2_night_mins    8.00  Num   
 34         hc_lbl_cnt_account_length    8.00  Num   
 35     hc_tar_frq_rat_account_length    8.00  Num   
 36       cpy_nom_mode_imp_lab_var_1_    8.00  Num   
 37         lchehi_lab_custserv_calls    8.00  Num   
 38              hc_lbl_cnt_day_calls    8.00  Num   
 39          hc_tar_frq_rat_day_calls    8.00  Num   
 40              hc_lbl_cnt_eve_calls    8.00  Num   
 41          hc_tar_frq_rat_eve_calls    8.00  Num   
 42   cpy_nom_mode_imp_lab_intl_calls    8.00  Num   
 43             lchehi_lab_intl_calls    8.00  Num   
 44            hc_l

In [135]:
sess.score(table='CHURN',casout=dict(name='fm_result', caslib='casuser', replace =True),rstore=churn_model_fm_, copyvars=['CHURN'])

[OutputCasTables]

              casLib       Name  Rows  Columns  \
 0  CASUSER(sasdemo)  fm_result  3333       51   
 
                                            casTable  
 0  CASTable('fm_result', caslib='CASUSER(sasdemo)')  

[Timing]

 Task Timing
 
                  Task  Seconds  Percent
 0   Loading the Store     0.00     0.01
 1  Creating the State     0.01     0.30
 2             Scoring     0.02     0.68
 3               Total     0.02     1.00

+ Elapsed: 0.0275s, user: 0.0356s, sys: 0.0221s, mem: 57.4mb

In [136]:
fm_result = sess.CASTable('fm_result')

In [137]:
fm_result.head()

,nhoks_nloks_dtree_10_day_charge,nhoks_nloks_dtree_5_day_charge,nhoks_nloks_log_day_charge,nhoks_nloks_pow_n0_5_day_charge,nhoks_nloks_dtree_10_day_mins,nhoks_nloks_dtree_5_day_mins,nhoks_nloks_log_day_mins,nhoks_nloks_pow_p0_5_day_mins,nhoks_nloks_dtree_10_eve_charge,nhoks_nloks_dtree_5_eve_charge,...,hc_tar_frq_rat_eve_calls,cpy_nom_mode_imp_lab_intl_calls,lchehi_lab_intl_calls,hc_lbl_cnt_night_calls,hc_tar_frq_rat_night_calls,lcnhenhi_dtree10_vmail_message,lcnhenhi_dtree5_vmail_message,cpy_nom_mode_imp_lab_INTL_PLAN,cpy_nom_mode_imp_lab_VMAIL_PLAN,CHURN
0,9.00,4.00,3.83,0.15,9.00,4.00,5.58,16.31,5.00,3.00,...,0.88,4.00,4.00,3.00,0.88,1.00,1.00,1.00,2.00,0
1,4.00,1.00,3.35,0.19,4.00,1.00,5.09,12.75,5.00,3.00,...,0.88,4.00,4.00,9.00,0.88,7.00,4.00,1.00,2.00,0
2,8.00,3.00,3.75,0.15,8.00,3.00,5.50,15.63,1.00,1.00,...,0.79,6.00,6.00,2.00,0.87,8.00,4.00,1.00,1.00,0
3,10.00,5.00,3.95,0.14,10.00,5.00,5.71,17.33,1.00,1.00,...,0.84,8.00,8.00,22.00,0.85,8.00,4.00,2.00,1.00,0
4,4.00,1.00,3.38,0.18,4.00,1.00,5.12,12.95,2.00,2.00,...,0.76,4.00,4.00,38.00,0.84,8.00,4.00,2.00,1.00,0


In [138]:
pipeline_out = sess.CASTable('PIPELINE_OUT')
pipeline_out.head(10)

,PipelineId,ModelType,MLType,Objective,ObjectiveType,Target,NFeatures,Feat1Id,Feat1IsNom,Feat2Id,...,Feat17Id,Feat17IsNom,Feat18Id,Feat18IsNom,Feat19Id,Feat19IsNom,Feat20Id,Feat20IsNom,Feat21Id,Feat21IsNom
0,2.00,binary classification,gradBoost,0.93,AUC,CHURN,21.00,105.00,1.00,85.00,...,94.00,0.00,93.00,1.00,95.00,1.00,63.00,1.00,73.00,1.00
1,18.00,binary classification,gradBoost,0.93,AUC,CHURN,20.00,105.00,1.00,85.00,...,94.00,0.00,93.00,1.00,95.00,1.00,67.00,0.00,nan,nan
2,4.00,binary classification,gradBoost,0.92,AUC,CHURN,16.00,105.00,1.00,85.00,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,12.00,binary classification,gradBoost,0.91,AUC,CHURN,12.00,105.00,1.00,86.00,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,20.00,binary classification,gradBoost,0.91,AUC,CHURN,12.00,105.00,1.00,86.00,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
5,8.00,binary classification,gradBoost,0.90,AUC,CHURN,10.00,105.00,1.00,85.00,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
6,14.00,binary classification,gradBoost,0.90,AUC,CHURN,10.00,105.00,1.00,85.00,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
7,16.00,binary classification,gradBoost,0.90,AUC,CHURN,12.00,105.00,1.00,85.00,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
8,6.00,binary classification,gradBoost,0.90,AUC,CHURN,14.00,105.00,1.00,85.00,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
9,19.00,binary classification,dtree,0.89,AUC,CHURN,12.00,105.00,1.00,86.00,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [139]:
champ_model = sess.CASTable('CHURN_MODEL_GRADBOOST_1')

sess.score(table=fm_result,
           casout=dict(name='model_result', caslib='casuser', replace =True),
           rstore=champ_model)

[OutputCasTables]

              casLib          Name  Rows  Columns  \
 0  CASUSER(sasdemo)  model_result  3333        4   
 
                                             casTable  
 0  CASTable('model_result', caslib='CASUSER(sasde...  

[Timing]

 Task Timing
 
                  Task  Seconds  Percent
 0   Loading the Store     0.00     0.00
 1  Creating the State     0.06     0.76
 2             Scoring     0.02     0.23
 3               Total     0.08     1.00

+ Elapsed: 0.0894s, user: 0.206s, sys: 0.00406s, mem: 57.3mb

In [148]:
sess.save(table=champ_model,name='automl_churn_champ.sashdat', replace=True)

NOTE: Cloud Analytic Services saved the file automl_churn_champ.sashdat in caslib CASUSER(sasdemo).


[caslib]

 'CASUSER(sasdemo)'

[name]

 'automl_churn_champ.sashdat'

+ Elapsed: 0.0185s, user: 0.00361s, sys: 0.00593s, mem: 16.5mb

In [140]:
result = sess.CASTable('model_result')
result.head()

,P_CHURN0,P_CHURN1,I_CHURN,_WARN_
0,0.97,0.03,0,
1,0.97,0.03,0,
2,0.93,0.07,0,
3,0.80,0.20,0,
4,0.93,0.07,0,


In [150]:
sess.save(table=result,name='model_scoring_result.sashdat', replace=True)

NOTE: Cloud Analytic Services saved the file model_scoring_result.sashdat in caslib CASUSER(sasdemo).


[caslib]

 'CASUSER(sasdemo)'

[name]

 'model_scoring_result.sashdat'

+ Elapsed: 0.00796s, user: 0.00326s, sys: 0.000887s, mem: 16.5mb

In [153]:
sess.fileinfo(allfiles=True)

,Permission,Owner,Group,Name,Size,Encryption,Time,ModTime
0,-rwxr-xr-x,sasdemo,sas,automl_churn_champ.sashdat,1961312,NONE,2020-05-30T00:31:49+00:00,1906417909.48
1,-rwxr-xr-x,sasdemo,sas,model_scoring_result.sashdat,197592,NONE,2020-05-30T00:33:20+00:00,1906417999.82


In [84]:
sess.close()

In [156]:
import h2o
from h2o.automl import H2OAutoML
h2o.init('http://ccbu-vidk.dlviyacluster.sashq-d.openstack.sas.com:54321/')

Checking whether there is an H2O instance running at http://ccbu-vidk.dlviyacluster.sashq-d.openstack.sas.com:54321 . connected.


H2O_cluster_uptime:,12 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.2
H2O_cluster_version_age:,1 month
H2O_cluster_name:,root
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,942 Mb
H2O_cluster_total_cores:,24
H2O_cluster_allowed_cores:,24
H2O_cluster_status:,"accepting new members, healthy"


In [157]:
churn_h2o = h2o.H2OFrame(out.to_frame())

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [158]:
churn_h2o.head()

account_length,vmail_message,day_mins,eve_mins,night_mins,intl_mins,custserv_calls,day_calls,day_charge,eve_calls,eve_charge,night_calls,night_charge,intl_calls,intl_charge,state,phone,INTL_PLAN,VMAIL_PLAN,AREA_CODE,CHURN
128,25,265.1,197.4,244.7,10,1,110,45.07,99,16.78,91,11.01,3,2.7,KS,382-4657,0,1,415,0
107,26,161.6,195.5,254.4,13.7,1,123,27.47,103,16.62,103,11.45,3,3.7,OH,371-7191,0,1,415,0
137,0,243.4,121.2,162.6,12.2,0,114,41.38,110,10.3,104,7.32,5,3.29,NJ,358-1921,0,0,415,0
84,0,299.4,61.9,196.9,6.6,2,71,50.9,88,5.26,89,8.86,7,1.78,OH,375-9999,1,0,408,0
75,0,166.7,148.3,186.9,10.1,3,113,28.34,122,12.61,121,8.41,3,2.73,OK,330-6626,1,0,415,0
118,0,223.4,220.6,203.9,6.3,0,98,37.98,101,18.75,118,9.18,6,1.7,AL,391-8027,1,0,510,0
121,24,218.2,348.5,212.6,7.5,3,88,37.09,108,29.62,118,9.57,7,2.03,MA,355-9993,0,1,510,0
147,0,157,103.1,211.8,7.1,0,79,26.69,94,8.76,96,9.53,6,1.92,MO,329-9001,1,0,415,0
117,0,184.5,351.6,215.8,8.7,1,97,31.37,80,29.89,90,9.71,4,2.35,LA,335-4719,0,0,408,0
141,37,258.6,222,326.4,11.2,0,84,43.96,111,18.87,97,14.69,5,3.02,WV,330-8173,1,1,415,0


In [159]:
x = churn_h2o.columns
y = 'CHURN'
x.remove(y)
x.remove('AREA_CODE')

In [160]:
churn_h2o[y] = churn_h2o[y].asfactor()

In [161]:
# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=20, seed=1,stopping_metric='auc',sort_metric='auc')
aml.train(x=x, y=y, training_frame=churn_h2o)

# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)

AutoML progress: |████████████████████████████████████████████████████████| 100%


model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
XGBoost_grid__1_AutoML_20200530_003603_model_4,0.919038,0.170703,0.858957,0.122468,0.20374,0.04151
XGBoost_grid__1_AutoML_20200530_003603_model_3,0.916229,0.182804,0.859997,0.121415,0.211499,0.044732
XGBoost_3_AutoML_20200530_003603,0.914528,0.171909,0.858757,0.132592,0.203327,0.041342
XGBoost_grid__1_AutoML_20200530_003603_model_1,0.913543,0.188334,0.837277,0.117239,0.216441,0.0468466
StackedEnsemble_AllModels_AutoML_20200530_003603,0.913019,0.148907,0.874923,0.1078,0.190832,0.0364169
XGBoost_1_AutoML_20200530_003603,0.912105,0.185976,0.846952,0.12424,0.2134,0.0455396
StackedEnsemble_BestOfFamily_AutoML_20200530_003603,0.911453,0.154315,0.869309,0.105431,0.194293,0.0377497
XGBoost_2_AutoML_20200530_003603,0.910856,0.205302,0.817516,0.135697,0.23115,0.0534304
GBM_grid__1_AutoML_20200530_003603_model_2,0.907632,0.228659,0.814674,0.147225,0.24948,0.0622404
XGBoost_grid__1_AutoML_20200530_003603_model_2,0.907014,0.24491,0.852866,0.124363,0.21243,0.0451265


In [162]:
h2o.shutdown()

/home/centos/.conda/envs/dl4viya/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  """Entry point for launching an IPython kernel.


H2O session _sid_92b0 closed.
